## Preparation
- Create output directory
- define visualization function

In [1]:
from phyloshape import set_log_level
set_log_level("DEBUG")
import os
output_dir = 'Gesneriaceae-out'
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

🐞 2023-05-09-16:46:20.29 | logger_setup.py | set_log_level             | phyloshape v.0.0.1 logging enabled


In [2]:
def visualize_points(*input_points, **kwargs):
    plot = k3d.plot(grid_visible=False) #,
                #camera_auto_fit=False)
    offset_v = np.array([0, 0, 0])
    for _points in input_points:
        plt_points = k3d.points(_points + offset_v,
                                point_size=kwargs.get("point_size", 20),
                                shader="flat")
        plot += plt_points
        offset_v[0] = offset_v[0] + kwargs.get("xgap", 2000)
    return plot

In [3]:
# PCA transformator
from sklearn.decomposition import PCA

class HDTrans:
    
    def __init__(self):
        self.sample_data = None
        self.sample_size = None
        self.each_s_shape = None
        self.maxs = None
        self.sample_normalized = None
        self.pca = None
        
    def transform(self, sample_data):
        if self.sample_data is None:
            self.sample_data = sample_data
            self.sample_size = len(self.sample_data)
            self.each_s_shape = self.sample_data[0].shape
            # Normalize data to [0,1] intervals. Supply the scale factor or
            # compute the maximum value among all the samples.
            self.maxs = np.max(sample_data)
            self.sample_normalized = np.array([np.ravel(s) / self.maxs for s in sample_data])
        if self.pca is None:
            self.pca = PCA()
            self.pca.fit(self.sample_normalized)
        return self.pca.transform(self.sample_normalized)
    
    def inverse_transform(self, transformed_data_per_sample):
        reconstructed_sample = self.pca.inverse_transform(transformed_data_per_sample)
        reconstructed_sample *= self.maxs
        return reconstructed_sample.reshape(self.each_s_shape)

## Extract the landmarks

The file `../data/Gesneriaceae.Gigascience.2020/Landmark_description_rev_1205.csv` with header contains four columns, with first column being the landmark value orders (1-based index) with the string `row` at the beginning of each value, the second column being the value type (`x`, `y`, or `z`), the third being the landmark description. 

The landmark values of each sample is store in the headless csv files matching `r'\d+_[A-Z]+\d+-?\w*_\d+\.csv'` under `../data/Gesneriaceae.Gigascience.2020-samples/`. Each file stores the landmark values in a single line splitted by `,` (or `\t`) in the order recorded in `Landmark_description_rev_1205.csv`.

In [4]:
# import csv
# import numpy as np
# import re

# # Function to determine the file format
# def get_delimiter(filename):
#     with open(filename, 'r') as f:
#         sample = f.read(1024)
#         sniffer = csv.Sniffer()
#         dialect = sniffer.sniff(sample)
#         return dialect.delimiter

# # Read the landmark description file
# type_to_id = {'x': 0, 'y': 1, 'z': 2}
# with open('../data/Gesneriaceae.Gigascience.2020/Landmark_description_rev_1205.csv', 'r') as f:
#     reader = csv.reader(f)
#     next(reader)  # skip header
#     landmarks_lines = [row for row in reader]
#     landmarks = np.zeros((len(landmarks_lines)//3, 3), dtype=np.int64)
#     describe_to_row = {}
#     for i, (order, value_type, describe, _) in enumerate(landmarks_lines):
#         order = int(order[3:]) - 1  # convert to 0-based index
#         if describe in describe_to_row:
#             row = describe_to_row[describe]
#         else:
#             row = describe_to_row[describe] = order
#         landmarks[row, type_to_id[value_type]] = order
        
# # Process each sample file
# sample_dir = '../data/Gesneriaceae.Gigascience.2020/'
# pattern = r'\d+_[A-Z]+\d+-?\w*_\d+\.csv'
# counter = 0
# samples = {}
# for filename in os.listdir(sample_dir):
#     if re.match(pattern, filename):
#         filepath = os.path.join(sample_dir, filename)
#         delimiter = get_delimiter(filepath)
#         if delimiter == '\t':
#             print(filepath + ' is actually TSV!')
#         with open(filepath, 'r') as f:
#             # all the sample files have the .csv extension but some of them are actually in TSV format!!!
#             reader = csv.reader(f, delimiter=delimiter)
#             values = next(reader)
#             samples[filename[:-4]] = np.take(values, landmarks).astype(np.float64)
#         # Save the result to a new csv file
#         np.savetxt(os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_landmarks.tab'), samples[filename[:-4]], delimiter='\t', fmt='%.6f')
#         # with open(os.path.join(output_dir, f'{os.path.splitext(filename)[0]}_landmarks.csv'), 'w') as f:
#         #     writer = csv.writer(f)
#         #     writer.writerows(result)
#         counter += 1
# print(f'{counter} samples extracted!')

In [5]:
import numpy as np
samples = {}
for filename in os.listdir(output_dir):
    if filename.endswith("_landmarks.tab"):
        sample_n = filename.replace("_landmarks.tab", "")
        samples[sample_n] = np.loadtxt(os.path.join(output_dir, filename), dtype=np.float64, delimiter="\t")

#### Use extracted instead

## Plot the point cloud

In [6]:
import k3d
from phyloshape import *
from phyloshape.shape.src.vectors import VertexVectorMapper

In [7]:
len(samples)

152

In [8]:
# test_sample = '07_K039091_08'
test_sample = '12_K039105_04'

In [9]:
visualize_points(samples[test_sample].astype(np.float32))
# 123 is the outlier, described as Left_dorsal_lobe_midrib_proximal_end_primary_landmark/Left_dorsal_tube_midrib_distal_end_primary_landmark

/home/jianjun/mambaforge/envs/phyloshape/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

## test the vertices-based system mapper

In [10]:
# test_sample = '07_K039091_08'
# test_sample = '58_HC5803-7_09'
alignment = ShapeAlignment()
for s_n, s_v in samples.items():
    alignment.append(label=s_n, sample=Vertices(coords=s_v))

In [11]:
s_label, s_vertices = alignment[test_sample]
len(s_vertices)

420

In [12]:
alignment.deduplicate()

ℹ️ 2023-05-09-16:46:21.09 |        shape.py | find_duplicate            | 411 ouf of 420 sample-wide unique points


In [13]:
# 123 is the outlier, described as Left_dorsal_lobe_midrib_proximal_end_primary_landmark/Left_dorsal_tube_midrib_distal_end_primary_landmark
alignment.del_vertices(123)

In [14]:
s_label, s_vertices = alignment[test_sample]
len(s_vertices)

410

In [15]:
visualize_points(s_vertices.coords)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [16]:
# works in old system
from phyloshape.shape.src.vectors import VertexVectorMapperOld
old_mapper = VertexVectorMapperOld(s_vertices.coords)
old_new_vs = old_mapper.to_vertices(old_mapper.to_vectors(s_vertices.coords))
visualize_points(old_new_vs)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [17]:
mapper = VertexVectorMapper(alignment.get_vertices_list(), mode="linear-variation")  # linear-variation, linear-random, network-local

ℹ️ 2023-05-09-16:46:21.30 |      vectors.py | check_duplicates          | 410 ouf of 410 sample-wide unique points


In [18]:
len(mapper._vh_bundle_list)

419

In [19]:
# this_coords = s_vertices.coords
# preorder_traverse = mapper.get_lines_for_k3d_plot()
# from phyloshape.utils import rgb_to_hex
# from colorsys import hsv_to_rgb
# HSV_vals = [(0.5, x*1.0/len(this_coords), x*1.0/len(this_coords)) for x in range(len(this_coords))]
# RGB_vals = np.array([hsv_to_rgb(*_hsv) for _hsv in HSV_vals])
# HEX_vals = rgb_to_hex(RGB_vals * 256)
# plot = k3d.plot()
# plot += k3d.points(this_coords, colors=HEX_vals[preorder_traverse[:len(HEX_vals)]], point_size=50)
# plot += k3d.line(this_coords[preorder_traverse][:4], shader="simple", width=.3, color=0xff0000)
# # for go_v, v_tri in enumerate(this_coords[preorder_traverse[:int(len(preorder_traverse)/2)]]):
# # plot += k3d.line(v_tri, shader="simple", width=200, color=0xff0000)# int(HEX_vals[go_v]))
# plot

In [20]:
vects = mapper.to_vectors(s_vertices.coords)
vects

array([[ 8.73249590e+01,  0.00000000e+00,  0.00000000e+00],
       [ 5.01472244e+01,  1.47084229e+02, -7.58112901e-06],
       [-3.71777344e+01,  1.47084229e+02, -5.00966871e-06],
       ...,
       [ 1.45454674e+01, -1.42705505e+02, -2.04595367e+02],
       [ 2.11604782e+02,  1.83102203e+02, -5.26193008e+01],
       [ 2.42806091e+02, -1.68711121e+02,  1.83234497e+02]])

In [21]:
# convert vects back to vertices
new_s = mapper.to_vertices(vects)

🐞 2023-05-09-16:46:28.67 |      vectors.py | __updating_vertices       | total num of iterations: 5


In [22]:
# mapper._debug_vertices

In [23]:
visualize_points(s_vertices.coords)  #  123 is the outlier

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [24]:
visualize_points(new_s)  # 0,123,279 were the outliers in the linear-variation mode with the variation summing bug

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

## Test Phyloshape using minimal tree

In [25]:
import csv
sample_to_species = {}
with open("../data/Gesneriaceae.Gigascience.2020/samples.manual_sum.tab") as read_sample_info:
    csv_reader = csv.reader(read_sample_info, delimiter='\t')
    next(csv_reader)  # skip header
    for sample_, species_, *foo in csv_reader:
        sample_to_species[sample_] = species_

In [26]:
minimal_tree_str = "((((19_K039112_04:0.001,19_K039112_05:0.001):0.013,63_K039178_01:0.018):0.002,58_HC5803-7_09:0.025):0.003,22_K039118_03:0.012);"

In [27]:
from phyloshape import ShapeAlignment, PhyloShape, Brownian
from toytree.io.src import newick

In [28]:
minimal_tree = newick.parse_newick_string(minimal_tree_str)
minimal_tree.draw(node_labels=True, node_sizes=15, tip_labels=[f"{sample_to_species[raw_tip]} ({raw_tip})" for raw_tip in minimal_tree.get_tip_labels()])

(<toyplot.canvas.Canvas at 0x7f450bf03e10>,
 <toytree.drawing.src.toytree_mark.ToytreeMark at 0x7f450bdf6790>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="450.0px" height="275.0px" viewBox="0 0 450.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te7e1ccc33eac4d40bbe78558b435b8fc"> 5 6 7 8 Sinningia carangolensis  (19_K039112_04) Sinningia carangolensis  (19_K039112_05) Sinningia sceptrum (63_K039178_01) Sinningia pusilla (58_HC5803-7_09) Sinningia concinna  (22_K039118_03)

In [29]:
alignment = ShapeAlignment()
for node_id in range(minimal_tree.ntips):
    s_n = minimal_tree[node_id].name
    alignment.append(label=s_n, sample=Vertices(coords=samples[s_n]))
alignment.deduplicate()

ℹ️ 2023-05-09-16:46:28.78 |        shape.py | find_duplicate            | 411 ouf of 420 sample-wide unique points


## Use Generalized Procrustes Analysis

In [30]:
# define the PCA transformer
hd_trans = HDTrans()
# define the tree
minimal_tree = newick.parse_newick_string(minimal_tree_str)
# define the phyloshape object
ps_gpa = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian(), dim_transform=hd_trans.transform, dim_inverse_transform=hd_trans.inverse_transform)
ps_gpa.reconstruct_ancestral_shapes_using_gpa(scale=True)

ℹ️ 2023-05-09-16:46:28.80 |        phylo.py | build_tip_moves           | Dimension (411, 3) -> (5,)
ℹ️ 2023-05-09-16:46:28.80 |        phylo.py | build_tip_moves           | Post-GPA moves for 5 tips assigned.
ℹ️ 2023-05-09-16:46:28.80 |        phylo.py | sym_ancestral_state       | Moves for 4 ancestral nodes symbolized.
ℹ️ 2023-05-09-16:46:29.14 |        phylo.py | formularize_log_like      | Num of variables: 21
ℹ️ 2023-05-09-16:46:29.14 |        phylo.py | formularize_log_like      | Log-likelihood formula constructed.
ℹ️ 2023-05-09-16:46:29.16 |        phylo.py | minimize_negloglike       | Searching for the best solution ..
ℹ️ 2023-05-09-16:46:29.25 |        phylo.py | minimize_negloglike       | Loglikelihood: 16.886199352346683
ℹ️ 2023-05-09-16:46:29.25 |        phylo.py | __summarize_ml_result     | sigma^2=169.306238


In [33]:
visualize_points(*[ps_gpa.tree[node_id].vertices.coords for node_id in range(ps_gpa.tree.nnodes)])

/home/jianjun/mambaforge/envs/phyloshape/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

### using the linear-variation with num_vs=100

In [34]:
num_vt_iter = 5
# define the PCA transformer
vector_trans = HDTrans()
# define the tree
minimal_tree = newick.parse_newick_string(minimal_tree_str)
# define the phyloshape object
ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian(), dim_transform=vector_trans.transform, dim_inverse_transform=vector_trans.inverse_transform)
# ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian())
ps.reconstruct_ancestral_shapes_using_ml(mode="linear-variation", num_vs=100, num_vt_iter=num_vt_iter)  # old, linear-variation, linear-random, network-local

ℹ️ 2023-05-09-16:47:24.62 |      vectors.py | check_duplicates          | 411 ouf of 411 sample-wide unique points
ℹ️ 2023-05-09-16:47:25.65 |        phylo.py | build_vv_translator       | Vertex:Vector (511:510) translator built.
ℹ️ 2023-05-09-16:47:43.47 |        phylo.py | build_tip_vectors         | Dimension (46050, 3) -> (5,)
ℹ️ 2023-05-09-16:47:43.47 |        phylo.py | build_tip_vectors         | Vectors for 5 tips built.
ℹ️ 2023-05-09-16:47:43.47 |        phylo.py | sym_ancestral_state       | Vectors for 4 ancestral nodes symbolized.
ℹ️ 2023-05-09-16:47:43.47 |        phylo.py | formularize_log_like      | Num of variables: 21
ℹ️ 2023-05-09-16:47:43.47 |        phylo.py | formularize_log_like      | Log-likelihood formula constructed.
ℹ️ 2023-05-09-16:47:43.49 |        phylo.py | minimize_negloglike       | Searching for the best solution ..
ℹ️ 2023-05-09-16:47:43.58 |        phylo.py | minimize_negloglike       | Loglikelihood: 94.4574282983511
ℹ️ 2023-05-09-16:47:43.58 |   

In [35]:
diffs = []
for go_iter in range(1, num_vt_iter):
    diff = np.linalg.norm(ps.vv_translator._debug_vertices[go_iter]-ps.vv_translator._debug_vertices[go_iter-1], axis=1)
    diffs.append([np.mean(diff), np.std(diff)])
diffs

[[392.35211965913936, 173.10776992656653],
 [27.934014298883667, 37.56855898555182],
 [11.4511394067966, 1.5801692522866366],
 [11.708944778358065, 1.4638768204822996]]

In [36]:
visualize_points(ps.tree[8].vertices.coords, point_size=20)

/home/jianjun/mambaforge/envs/phyloshape/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [37]:
minimal_tree.draw(node_labels=True, node_sizes=15, tip_labels=[f"{sample_to_species[raw_tip]} ({raw_tip})" for raw_tip in minimal_tree.get_tip_labels()])

(<toyplot.canvas.Canvas at 0x7f4507ffdc50>,
 <toytree.drawing.src.toytree_mark.ToytreeMark at 0x7f4507f09d10>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="450.0px" height="275.0px" viewBox="0 0 450.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3640faa9e7834a06b665508e1331f679"> 5 6 7 8 Sinningia carangolensis  (19_K039112_04) Sinningia carangolensis  (19_K039112_05) Sinningia sceptrum (63_K039178_01) Sinningia pusilla (58_HC5803-7_09) Sinningia concinna  (22_K039118_03)

In [38]:
# visualize_points(ps.tree[8].vertices.coords, point_size=20)
visualize_points(*[ps.tree[node_id].vertices.coords for node_id in range(ps.tree.nnodes)])

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [39]:
# # visualize the order of vertices to locate and highlight the first 4 vertices with red lines
# this_coords = ps.tree[8].vertices.coords
# preorder_traverse = ps.vv_translator.get_lines_for_k3d_plot()
# from phyloshape.utils import rgb_to_hex
# from colorsys import hsv_to_rgb
# HSV_vals = [(0.5, x*1.0/len(this_coords), x*1.0/len(this_coords)) for x in range(len(this_coords))]
# RGB_vals = np.array([hsv_to_rgb(*_hsv) for _hsv in HSV_vals])
# HEX_vals = rgb_to_hex(RGB_vals * 256)
# plot = k3d.plot(grid_visible=False)
# plot += k3d.points(this_coords, colors=HEX_vals[preorder_traverse[:len(HEX_vals)]], point_size=10)
# plot += k3d.line(this_coords[preorder_traverse][:4], shader="simple", width=.3, color=0xff0000)
# # for go_v, v_tri in enumerate(this_coords[preorder_traverse[:int(len(preorder_traverse)/2)]]):
# # plot += k3d.line(v_tri, shader="simple", width=200, color=0xff0000)# int(HEX_vals[go_v]))
# plot

In [40]:
visualize_points(ps_lv_100.tree[8].vertices.coords, point_size=20)

NameError: name 'ps_lv_100' is not defined

In [37]:
# # define the PCA tranformer
# vector_trans = VectorTrans()
# # define the tree
# minimal_tree = newick.parse_newick_string(minimal_tree_str)
# # define the phyloshape object
# ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian(), vect_transform=vector_trans.transform, vect_inverse_transform=vector_trans.inverse_transform)
# # ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian())
# ps.reconstruct_ancestral_shapes_using_ml(mode="network-local", num_vs=10)  # old, linear-variation, linear-random, network-local

In [38]:
# visualize_points(ps.tree[8].vertices.coords, point_size=20)
# # visualize_points(*[ps.tree[node_id].vertices.coords for node_id in range(ps.tree.nnodes)])

In [39]:
# # define the PCA tranformer
# vector_trans = VectorTrans()
# # define the tree
# minimal_tree = newick.parse_newick_string(minimal_tree_str)
# # define the phyloshape object
# ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian(), vect_transform=vector_trans.transform, vect_inverse_transform=vector_trans.inverse_transform)
# # ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian())
# ps.reconstruct_ancestral_shapes_using_ml(mode="network-local", num_vs=40)  # old, linear-variation, linear-random, network-local

In [40]:
# visualize_points(ps.tree[8].vertices.coords, point_size=20)
# # visualize_points(*[ps.tree[node_id].vertices.coords for node_id in range(ps.tree.nnodes)])

In [56]:
# define the PCA tranformer
num_vt_iter = 30
vector_trans = VectorTrans()
# define the tree
minimal_tree = newick.parse_newick_string(minimal_tree_str)
# define the phyloshape object
ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian(), vect_transform=vector_trans.transform, vect_inverse_transform=vector_trans.inverse_transform)
# ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian())
ps.reconstruct_ancestral_shapes_using_ml(mode="linear-random", num_vs=30, num_vt_iter=num_vt_iter)  # old, linear-variation, linear-random, network-local

ℹ️ 2023-05-04-00:29:25.69 |      vectors.py | check_duplicates          | 411 ouf of 411 sample-wide unique points
ℹ️ 2023-05-04-00:29:25.87 |        phylo.py | build_vv_translator       | Vertex:Vector (441:440) translator built.
ℹ️ 2023-05-04-00:29:30.83 |        phylo.py | build_tip_vectors         | Dimension (12765, 3) -> (5,)
ℹ️ 2023-05-04-00:29:30.83 |        phylo.py | build_tip_vectors         | Vectors for 5 tips built.
ℹ️ 2023-05-04-00:29:30.83 |        phylo.py | sym_ancestral_vectors     | Vectors for 4 ancestral nodes symbolized.
ℹ️ 2023-05-04-00:29:30.83 |        phylo.py | formularize_log_like      | Num of variables: 21
ℹ️ 2023-05-04-00:29:30.83 |        phylo.py | formularize_log_like      | Log-likelihood formula constructed.
ℹ️ 2023-05-04-00:29:30.85 |        phylo.py | minimize_negloglike       | Searching for the best solution ..
ℹ️ 2023-05-04-00:29:30.95 |        phylo.py | minimize_negloglike       | Loglikelihood: 61.17569979141426
ℹ️ 2023-05-04-00:29:30.96 |  

In [57]:
diffs = []
for go_iter in range(1, num_vt_iter):
    diff = np.linalg.norm(ps.vv_translator._debug_vertices[go_iter]-ps.vv_translator._debug_vertices[go_iter-1], axis=1)
    diffs.append([np.mean(diff), np.std(diff)])
diffs

[[296.4715887534307, 146.87178649582364],
 [30.158866358057963, 32.93018682576381],
 [22.53345697040081, 13.835607806922015],
 [18.94978388017831, 11.62198300187615],
 [17.211418351332295, 10.046742601879036],
 [15.524847812900632, 8.999592898327672],
 [16.95136893409433, 8.647499784063731],
 [17.96033765155797, 8.607769552691197],
 [20.005804002055633, 10.60097345238358],
 [22.201991722231096, 11.25641355812676],
 [22.5552948007904, 11.78248225038686],
 [22.983971722008366, 12.994112569355687],
 [26.25859975203253, 15.181743992976642],
 [25.95050232374893, 15.469570543697827],
 [22.46060316180184, 13.818646487787055],
 [20.619263768233946, 12.407172553842873],
 [20.31104562560389, 12.233320682292785],
 [19.979246148644098, 12.845776535828124],
 [19.026225371788648, 11.733648368814205],
 [20.399721017729263, 12.41011848161625],
 [23.217159454462195, 12.241447504998563],
 [24.511855345480424, 13.684720599754183],
 [26.208436483258176, 16.279315428985367],
 [20.95335942867157, 13.7913611

In [58]:
visualize_points(ps.tree[8].vertices.coords, point_size=20)
# visualize_points(*[ps.tree[node_id].vertices.coords for node_id in range(ps.tree.nnodes)])

/home/jianjun/mambaforge/envs/phyloshape/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [42]:
# define the PCA tranformer
num_vt_iter = 30
vector_trans = VectorTrans()
# define the tree
minimal_tree = newick.parse_newick_string(minimal_tree_str)
# define the phyloshape object
ps_lr_100 = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian(), vect_transform=vector_trans.transform, vect_inverse_transform=vector_trans.inverse_transform)
# ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian())
ps_lr_100.reconstruct_ancestral_shapes_using_ml(mode="linear-random", num_vs=100, num_vt_iter=num_vt_iter)  # old, linear-variation, linear-random, network-local

ℹ️ 2023-05-04-13:47:55.76 |      vectors.py | check_duplicates          | 411 ouf of 411 sample-wide unique points
ℹ️ 2023-05-04-13:47:56.56 |        phylo.py | build_vv_translator       | Vertex:Vector (511:510) translator built.
ℹ️ 2023-05-04-13:48:15.46 |        phylo.py | build_tip_vectors         | Dimension (46050, 3) -> (5,)
ℹ️ 2023-05-04-13:48:15.46 |        phylo.py | build_tip_vectors         | Vectors for 5 tips built.
ℹ️ 2023-05-04-13:48:15.46 |        phylo.py | sym_ancestral_vectors     | Vectors for 4 ancestral nodes symbolized.
ℹ️ 2023-05-04-13:48:15.47 |        phylo.py | formularize_log_like      | Num of variables: 21
ℹ️ 2023-05-04-13:48:15.47 |        phylo.py | formularize_log_like      | Log-likelihood formula constructed.
ℹ️ 2023-05-04-13:48:15.49 |        phylo.py | minimize_negloglike       | Searching for the best solution ..
ℹ️ 2023-05-04-13:48:15.58 |        phylo.py | minimize_negloglike       | Loglikelihood: 86.65566851079316
ℹ️ 2023-05-04-13:48:15.58 |  

In [57]:
diffs = []
for go_iter in range(1, num_vt_iter):
    diff = np.linalg.norm(ps_lr_100.vv_translator._debug_vertices[go_iter] - ps_lr_100.vv_translator._debug_vertices[go_iter-1], axis=1)
    diffs.append([np.max(diff), np.mean(diff), np.std(diff)])
diffs

[[671.659005421147, 275.5250063321256, 146.08720457940828],
 [380.7616190898165, 32.60258502028642, 51.438354623526415],
 [31.147314658855795, 14.338643216154118, 7.21004599643521],
 [31.490856405311174, 14.190523541385902, 7.22016032443849],
 [31.36492156264721, 14.221286949328576, 7.1444519386029555],
 [31.208214937172, 14.038536903441525, 7.266054104065067],
 [31.041529992257082, 13.677503731849562, 7.219166849642793],
 [29.778472162898467, 13.497687758524902, 6.873174557025921],
 [29.848147131279443, 13.449346144081247, 6.8127162684067155],
 [29.305949227768497, 13.238829882567222, 6.679543008876783],
 [29.339700426996295, 13.288594066569788, 6.639939175913306],
 [30.143657323163577, 13.232739280362676, 6.557921438329172],
 [29.167836333625687, 13.306371309901742, 6.442020600841703],
 [28.86210709114569, 13.305017277375075, 6.470899241523502],
 [28.791395172516186, 13.251970759119146, 6.4793539882241395],
 [30.16281692775791, 13.61474577549206, 6.682182782865429],
 [30.196843363970

In [58]:
diffs = []
for go_iter in range(1, num_vt_iter):
    ref_c, new_v = unscaled_procrustes(ps_lr_100.vv_translator._debug_vertices[go_iter], ps_lr_100.vv_translator._debug_vertices[go_iter-1])
    diff = np.linalg.norm(ref_c-new_v, axis=1)
    diffs.append([np.max(diff), np.mean(diff), np.std(diff)])
diffs

[[721.6393912764127, 268.31669411729104, 142.88302239182232],
 [381.66518984045695, 23.609664978664462, 54.384047529222926],
 [1.8709998089237896, 0.5374772195073566, 0.30481144835201346],
 [2.67231938535895, 0.5564194225677377, 0.3565781779537669],
 [1.8476317967331317, 0.5486954880284286, 0.3278923901200836],
 [2.576259220926404, 0.5867953608899107, 0.34279201008302174],
 [2.036117241554573, 0.5773746135712874, 0.3173294837782477],
 [2.4249957710867283, 0.5542151064868988, 0.32745539503800153],
 [1.9335698785226942, 0.48563719084139517, 0.3165421111504169],
 [2.5848296522826435, 0.5433306701847141, 0.30957134428846866],
 [1.8701491615514754, 0.5259759237774804, 0.32664116066098153],
 [2.185215015087066, 0.49170956151450906, 0.31605002460542486],
 [2.423289749520147, 0.4468245824130567, 0.3145276817457417],
 [1.9820334443045444, 0.5139033770680709, 0.30754952261109353],
 [2.3125729173551357, 0.532136245377543, 0.3389595721543202],
 [2.508862322700803, 0.5226356804056026, 0.35604506800

In [56]:
from scipy.spatial import procrustes
from scipy.linalg import orthogonal_procrustes
def unscaled_procrustes(reference, data):
    """Fit `data` to `reference` using procrustes analysis without scaling.
    Uses translation (mean-centering), reflection, and orthogonal rotation.

    Parameters:
        reference (array-like of shape (n_points, n_dim)): reference shape to
            fit `data` to
        data (array-like of shape (n_points, n_dim)): shape to align to
            `reference`

    Returns:
        reference_centered (np.ndarray of shape (n_points, n_dim)): 0-centered
            `reference` shape
        data_aligned (np.ndarray of shape (n_points, n_dim)): `data` aligned to
            the reference shape
    """
    # Convert inputs to np.ndarray types
    reference = np.array(reference, dtype=np.double)
    data = np.array(data, dtype=np.double)

    # Translate data to the origin
    reference_centered = reference - reference.mean(axis=0)
    data_centered = data - data.mean(axis=0)

    # Rotate / reflect data to match reference
    # transform mtx2 to minimize disparity
    R, _ = orthogonal_procrustes(data_centered, reference_centered)
    data_aligned = data_centered @ R

    return reference_centered, data_aligned

In [ ]:
visualize_points(ps_lr_100.tree[8].vertices.coords, point_size=20)
# visualize_points(*[ps.tree[node_id].vertices.coords for node_id in range(ps.tree.nnodes)])

In [54]:
sample_names = list(samples)[:10]
sample_names, [sample_to_species[s_] for s_ in sample_names]

(['63_K039178_01',
  '58_HC5803-7_09',
  '19_K039112_05',
  '19_K039112_02',
  '22_K039118_03',
  '69_K039199_02',
  '19_HC1912-2_03',
  '34_K039135_01',
  '63_K039179_06',
  '72_K039216_01'],
 ['Sinningia sceptrum',
  'Sinningia pusilla',
  'Sinningia carangolensis\xa0',
  'Sinningia carangolensis\xa0',
  'Sinningia concinna\xa0',
  'Sinningia tubiflora\xa0',
  'Sinningia carangolensis\xa0',
  'Sinningia harleyi\xa0',
  'Sinningia sceptrum',
  'Sinningia\xa0warmingii'])

In [55]:
bigger_tree_str = "((((19_K039112_05:0.001,19_K039112_05:0.001):0.013,63_K039178_01:0.018):0.002,58_HC5803-7_09:0.025):0.003,22_K039118_03:0.012);"

In [ ]:
alignment = ShapeAlignment()
for s_n in sample_names:
    alignment.append(label=s_n, sample=Vertices(coords=samples[s_n]))
alignment.deduplicate()

In [ ]:
minimal_tree = newick.parse_newick_string(minimal_tree_str)
minimal_tree.draw(node_labels=True, node_sizes=15, tip_labels=[f"{sample_to_species[raw_tip]} ({raw_tip})" for raw_tip in minimal_tree.get_tip_labels()])

In [ ]:
# define the PCA tranformer
vector_trans = VectorTrans()
# define the tree
minimal_tree = newick.parse_newick_string(minimal_tree_str)
# define the phyloshape object
ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian(), vect_transform=vector_trans.transform, vect_inverse_transform=vector_trans.inverse_transform)
# ps = PhyloShape(tree_obj=minimal_tree, shape_alignments=alignment, model=Brownian())
ps.reconstruct_ancestral_shapes_using_ml(mode="network-local", num_vs=10)  # old, linear-variation, linear-random, network-local

In [38]:
pcs.shape

(5, 5)

In [53]:
pca.inverse_transform(pcs).shape

(5, 300)

In [51]:
pca_test_samples[0][:3]

array([[ 559.14,  557.37, 1201.4 ],
       [ 537.25,  564.05, 1213.1 ],
       [ 519.59,  555.64, 1227.3 ]])

In [44]:
pca_test_samples_norm.shape

(5, 300)

In [145]:
pca_test_samples[0].shape

(100, 3)

In [68]:
pca_test_samples = [samples[_sample][:100] for _sample in sample_names]

In [71]:
pca_test_samples_norm = normalize(pca_test_samples)

In [159]:
pcs, m, s, T, u = compute_pca(pca_test_samples_norm)

In [169]:
pca_test_samples_norm

array([[0.44414965, 0.44274367, 0.9543252 , ..., 0.25795536, 0.43818413,
        0.80490905],
       [0.45080626, 0.45965525, 0.2781317 , ..., 0.34907459, 0.42230519,
        0.34060688],
       [0.33838272, 0.32957344, 0.8084836 , ..., 0.45198983, 0.49666375,
        0.8041147 ],
       [0.48565414, 0.3571769 , 0.82738899, ..., 0.29383589, 0.41761061,
        0.8161093 ],
       [0.29632219, 0.571769  , 0.36247518, ..., 0.41456827, 0.43832711,
        0.47864008]])

In [168]:
reconstruct(w=u, X=pcs, m=m, dim=300)

array([[0.42586099, 0.49005968, 0.75101113, ..., 0.28704397, 0.43720487,
        0.64350934],
       [0.46415803, 0.40648238, 0.56270978, ..., 0.33338396, 0.4287295 ,
        0.56982457],
       [0.37910995, 0.37073423, 0.65113822, ..., 0.42198881, 0.47089356,
        0.67078719],
       [0.51209961, 0.39964492, 0.55747818, ..., 0.29754007, 0.36331486,
        0.72708987],
       [0.39371349, 0.50286522, 0.54884025, ..., 0.34543653, 0.40569853,
        0.6509054 ]])

In [135]:
pca_test_samples = [[1, 2, 3, 5], [3, 4, 6, 7],[1, 2, 3, 5]]
pca_test_samples_norm = normalize(pca_test_samples)
pcs, m, s, T, u = compute_pca(pca_test_samples_norm)

In [136]:
pcs

array([[ 0.43643578,  0.43643578,  0.65465367,  0.43643578],
       [ 0.30367585,  0.03795948,  0.07591896, -0.94898702],
       [-0.25819889, -0.51639778, -0.77459667, -0.25819889]])

In [137]:
m

array([0.23809524, 0.38095238, 0.57142857, 0.80952381])

In [70]:
#! /usr/bin/env python
"""
Author: Jeremy M. Stober
Program: PCA.PY
Date: Tuesday, March 30 2010
Description: Principle component analysis.
"""

import numpy as np

# Note that PCA implementation in MDP cannot handle high dimensional
# data (at least the version I tested). Using a tricky set of
# convenient transformations, PCA can be applied to very high
# dimensional data as in this implementation.

def compute_pca(data):
    m = np.mean(data, axis=0)
    datac = np.array([obs - m for obs in data])
    T = np.dot(datac, datac.T)
    [u,s,v] = np.linalg.svd(T)

    # here iteration is over rows but the columns are the eigenvectors of T
    pcs = [np.dot(datac.T, item) for item in u.T ]

    # note that the eigenvectors are not normed after multiplication by T^T
    pcs = np.array([d / np.linalg.norm(d) for d in pcs])

    return pcs, m, s, T, u

def compute_projections(I,pcs,m):
    projections = []
    for i in I:
        w = []
        for p in pcs:
            w.append(np.dot(i - m, p))
        projections.append(w)
    return projections

def reconstruct(w, X, m,dim = 5):
    return np.dot(w[:dim],X[:dim,:]) + m

def normalize(samples, maxs = None):
    # Normalize data to [0,1] intervals. Supply the scale factor or
    # compute the maximum value among all the samples.

    if not maxs:
        maxs = np.max(samples)
    return np.array([np.ravel(s) / maxs for s in samples])